In [1]:
# Deliverable 1: Retrieve Weather Data
# -------------------------------------------------------------------
# Generate a set of 2,000 random latitudes and longitudes, retrieve the nearest city, and perform an API call 
# with the OpenWeatherMap. In addition to the city weather data you gathered in this module, use your API skills 
# to retrieve the current weather description for each city. Then, create a new DataFrame containing the updated 
# weather data.


import numpy as np

from citipy import citipy

from config_weather import weather_api_key 

import requests

import time

import pandas as pd

In [2]:
#Create a new set of 2,000 random latitudes and longitudes.

lat = np.random.uniform(-90,90, 2000)
lng = np.random.uniform(-180,180, 2000)

lat_lng = zip(lat,lng) #pairs the array

coordinates = list(lat_lng) #unzipping zipped tuple into a list

coordinates

[(58.363764647112504, 70.1833157372883),
 (3.5068040199480066, -117.90315645806572),
 (85.38160063271553, -112.08352857806827),
 (35.98694578440002, 76.66698271224595),
 (24.184897587882617, -63.45607485920148),
 (83.24723292363353, -21.071773526465677),
 (6.916539758588883, -138.2743214526813),
 (-79.96214623201041, 144.6726022330118),
 (-18.95349893809272, -133.32019362789987),
 (-27.434583623551454, -170.89529403957692),
 (86.18674722639048, 120.576895932497),
 (57.19799322076375, -135.76759033571557),
 (-55.149874004076594, 28.227328879404467),
 (-55.970035991986435, 66.08506926767461),
 (-47.83147420803782, 102.15793241313497),
 (37.07989541719, -147.29821927602376),
 (-83.8652650254591, 76.12240062300856),
 (-30.023419132293753, -105.46807998623274),
 (-43.590573052936676, 72.95183016337342),
 (-59.014191055303, 133.90203698471237),
 (20.53413870717648, 115.59210136843427),
 (-88.12226576521088, -145.60443575381632),
 (44.21138676978657, 123.69262751882923),
 (-56.136118376852394

In [6]:
#Get the nearest city using the citipy module.

cities = []

for row in coordinates:
    city = citipy.nearest_city(row[0], row[1]).city_name #row[0] is lat, #row[1] is long
    if city not in cities:
        cities.append(city)
    
cities


['ust-ishim',
 'san patricio',
 'yellowknife',
 'kargil',
 'road town',
 'illoqqortoormiut',
 'atuona',
 'hobart',
 'rikitea',
 'vaini',
 'saskylakh',
 'sitka',
 'port elizabeth',
 'saint-philippe',
 'busselton',
 'kahului',
 'mahebourg',
 'new norfolk',
 'jieshi',
 'mataura',
 'zhengjiatun',
 'avarua',
 'yumen',
 'labuhan',
 'cape town',
 'sinkat',
 'port alfred',
 'atbasar',
 'vaitupu',
 'punta arenas',
 'butler',
 'bambous virieux',
 'dikson',
 'mys shmidta',
 'westport',
 'castro',
 'mustasaari',
 'talnakh',
 'butaritari',
 'grindavik',
 'trelew',
 'bredasdorp',
 'faanui',
 'kapaa',
 'puerto ayora',
 'bethel',
 'manggar',
 'tuktoyaktuk',
 'klaksvik',
 'thompson',
 'georgetown',
 'lompoc',
 'tual',
 'east london',
 'chuy',
 'ancud',
 'jamestown',
 'fairbanks',
 'urucara',
 'geraldton',
 'guerrero negro',
 'taolanaro',
 'mount gambier',
 'kodiak',
 'khatanga',
 'ambilobe',
 'dehui',
 'souillac',
 'curanilahue',
 'kavieng',
 'winslow',
 'albany',
 'bluff',
 'kushima',
 'sedalia',
 'sa

In [4]:
# Perform an API call with the OpenWeatherMap.



#empty list to hold weather data
city_data = []

#print beginning of logging
print("Beginning of retrieval    ")
print("----------------------------")

#create counters
record_count = 1
set_count = 1

#loop through all cities
for i, city in enumerate(cities): # this lets us have both index and value #range(len(cities)) gives just index, for city in cities gives just city:
    #go through cities 50 at a time for logging purposes
    if (i % 50 == 0 and i >= 50): 
        set_count +=1
        record_count = 1 
        time.sleep(60) #tells program to stop for 60 seconds because API only lets us have 60 calls per minute on the free version to prevent time out errors
    #new city url for each city
    city_url = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid=" + weather_api_key + "&units=imperial" + "&q=" + city.replace(" ","+") #some city names have spaces, so we're taking them out and concatinating the names for the URL (otherwise it will find data for just the first part of the city name)
    print (f"processing record {record_count} of set {set_count} | {city}")
    #add 1 to record count
    record_count +=1

    #run API request on all the cities:
    try: 
        # Parse the JSON and retrieve data.
        weather_request=requests.get(city_url).json()
        
        #Parse out the needed data
        country = weather_request["sys"]["country"]
        latitude = weather_request["coord"]["lon"]
        longitude = weather_request["coord"]["lat"]
        max_temp = weather_request["main"]["temp_max"]
        humidity = weather_request["main"]["humidity"]
        cloudiness = weather_request["clouds"]["all"]
        wind_speed = weather_request["wind"]["speed"]
        weather_description = weather_request["weather"][0]["description"] #weather is a list of dictionaries, so look at the first row in the list (at index 0), and then the description key, with the "clear sky" value
        
        #adding data to city data list:
        city_data.append({ #appending data as a list of dictionaries
            "city":city.title(), #.title makes the first letter in each string capitalized
            "country":country,
            "latitude":latitude,
            "longitude":longitude,
            "max_temp":max_temp,
            "humidity":humidity,
            "cloudiness":cloudiness,
            "wind_speed":wind_speed,
            "weather_description":weather_description})
    #if an error comes up, skip the city
    except:
        print("City not found. Skipping...")
        pass #tells it that if city is not found, don't add previous city's data #pass is a general purpose statement to handle all errors encountered and to allow the program to continue
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    


Beginning of retrieval    
----------------------------
processing record 1 of set 1 | ust-ishim
processing record 2 of set 1 | san patricio
processing record 3 of set 1 | yellowknife
processing record 4 of set 1 | kargil
processing record 5 of set 1 | road town
processing record 6 of set 1 | illoqqortoormiut
City not found. Skipping...
processing record 7 of set 1 | atuona
processing record 8 of set 1 | hobart
processing record 9 of set 1 | rikitea
processing record 10 of set 1 | vaini
processing record 11 of set 1 | saskylakh
processing record 12 of set 1 | sitka
processing record 13 of set 1 | port elizabeth
processing record 14 of set 1 | saint-philippe
processing record 15 of set 1 | busselton
processing record 16 of set 1 | kahului
processing record 17 of set 1 | mahebourg
processing record 18 of set 1 | new norfolk
processing record 19 of set 1 | jieshi
processing record 20 of set 1 | mataura
processing record 21 of set 1 | zhengjiatun
processing record 22 of set 1 | avarua
proc

processing record 40 of set 4 | bilma
processing record 41 of set 4 | belushya guba
City not found. Skipping...
processing record 42 of set 4 | guaruja
processing record 43 of set 4 | oskarshamn
processing record 44 of set 4 | evensk
processing record 45 of set 4 | qaanaaq
processing record 46 of set 4 | manzil tamim
City not found. Skipping...
processing record 47 of set 4 | aojiang
processing record 48 of set 4 | coahuayana
processing record 49 of set 4 | iskateley
processing record 50 of set 4 | olean
processing record 1 of set 5 | marawi
processing record 2 of set 5 | arraial do cabo
processing record 3 of set 5 | ahtopol
processing record 4 of set 5 | boa vista
processing record 5 of set 5 | cururupu
processing record 6 of set 5 | severo-kurilsk
processing record 7 of set 5 | barrow
processing record 8 of set 5 | mega
processing record 9 of set 5 | tambong
processing record 10 of set 5 | vao
processing record 11 of set 5 | toora-khem
processing record 12 of set 5 | ternate
process

processing record 26 of set 8 | wajid
processing record 27 of set 8 | torbay
processing record 28 of set 8 | yemtsa
processing record 29 of set 8 | boyabat
processing record 30 of set 8 | wesselsbron
processing record 31 of set 8 | lolua
City not found. Skipping...
processing record 32 of set 8 | agadir
processing record 33 of set 8 | port shepstone
processing record 34 of set 8 | makueni
processing record 35 of set 8 | lasa
processing record 36 of set 8 | beyneu
processing record 37 of set 8 | thessalon
processing record 38 of set 8 | muskegon
processing record 39 of set 8 | kerteh
City not found. Skipping...
processing record 40 of set 8 | jalu
processing record 41 of set 8 | conde
processing record 42 of set 8 | morant bay
processing record 43 of set 8 | kyra
processing record 44 of set 8 | carmen del parana
processing record 45 of set 8 | owatonna
processing record 46 of set 8 | vestmanna
processing record 47 of set 8 | tucurui
processing record 48 of set 8 | brae
processing record

processing record 14 of set 12 | anar darreh
City not found. Skipping...
processing record 15 of set 12 | formoso do araguaia
City not found. Skipping...
processing record 16 of set 12 | trairi
processing record 17 of set 12 | vila franca do campo
processing record 18 of set 12 | flin flon
processing record 19 of set 12 | laiagam
City not found. Skipping...
processing record 20 of set 12 | verkhnyaya inta
processing record 21 of set 12 | port macquarie
processing record 22 of set 12 | morristown
processing record 23 of set 12 | pershotravneve
processing record 24 of set 12 | hirara
processing record 25 of set 12 | parole
processing record 26 of set 12 | jaitaran
processing record 27 of set 12 | skibbereen
processing record 28 of set 12 | lalmohan
processing record 29 of set 12 | tessalit
processing record 30 of set 12 | moindou
processing record 31 of set 12 | save
processing record 32 of set 12 | sangmelima
processing record 33 of set 12 | aguimes
processing record 34 of set 12 | moos

City not found. Skipping...
processing record 50 of set 15 | novoagansk
processing record 1 of set 16 | mahina
-----------------------------
Data Retrieval Complete      
-----------------------------


In [7]:
city_data

[{'city': 'Ust-Ishim',
  'country': 'RU',
  'latitude': 71.1665,
  'longitude': 57.6935,
  'max_temp': 51.84,
  'humidity': 52,
  'cloudiness': 92,
  'wind_speed': 10.2,
  'weather_description': 'overcast clouds'},
 {'city': 'San Patricio',
  'country': 'US',
  'latitude': -97.5169,
  'longitude': 28.017,
  'max_temp': 93.29,
  'humidity': 39,
  'cloudiness': 5,
  'wind_speed': 20.45,
  'weather_description': 'clear sky'},
 {'city': 'Yellowknife',
  'country': 'CA',
  'latitude': -114.3525,
  'longitude': 62.456,
  'max_temp': 72.12,
  'humidity': 28,
  'cloudiness': 75,
  'wind_speed': 12.66,
  'weather_description': 'broken clouds'},
 {'city': 'Kargil',
  'country': 'IN',
  'latitude': 76.1,
  'longitude': 34.5667,
  'max_temp': 51.75,
  'humidity': 46,
  'cloudiness': 20,
  'wind_speed': 6.42,
  'weather_description': 'few clouds'},
 {'city': 'Road Town',
  'country': 'VG',
  'latitude': -64.6167,
  'longitude': 18.4167,
  'max_temp': 86.04,
  'humidity': 66,
  'cloudiness': 40,
  '

In [8]:
#add weather to a new dataframe
city_df = pd.DataFrame(city_data)

city_df


,city,country,latitude,longitude,max_temp,humidity,cloudiness,wind_speed,weather_description
0,Ust-Ishim,RU,71.1665,57.6935,51.84,52,92,10.20,overcast clouds
1,San Patricio,US,-97.5169,28.0170,93.29,39,5,20.45,clear sky
2,Yellowknife,CA,-114.3525,62.4560,72.12,28,75,12.66,broken clouds
3,Kargil,IN,76.1000,34.5667,51.75,46,20,6.42,few clouds
4,Road Town,VG,-64.6167,18.4167,86.04,66,40,10.36,scattered clouds
...,...,...,...,...,...,...,...,...,...
682,Gillette,US,-105.5022,44.2911,55.11,71,100,5.75,overcast clouds
683,Sorland,NO,12.6934,67.6670,44.46,62,99,17.94,overcast clouds
684,Lubao,PH,120.5500,14.9167,82.11,87,99,4.00,light rain
685,Novoagansk,RU,76.6625,61.9449,42.22,64,6,5.39,clear sky


In [10]:
#Export the DataFrame as WeatherPy_Database.csv into the Weather_Database folder

file_name="WeatherPy_Database.csv"

city_df.to_csv(file_name, index_label="city_id")